# Make figures of dataset examples

In [ ]:
from avstack.config import Config
from fov.segmentation.utils import get_dataset


device = "cpu"

# set up datasets
dataset_cfgs = {
    "CARLA": "../../config/segmentation/__base__/datasets/carla.py",
    "nuScenes": "../../config/segmentation/__base__/datasets/nuscenes.py",
    "UGV": "../../config/segmentation/__base__/datasets/ugv.py",
    # "CARLA_adv": "../../config/segmentation/__base__/datasets/carla_adversarial.py",
    # "nuScenes_adv": "../../config/segmentation/__base__/datasets/nuscenes_adversarial.py",
    # "UGV_adv": "../../config/segmentation/__base__/datasets/ugv_adversarial.py",
}

# preload all the datasets (SM, dataset)
datasets = {
    d_name: get_dataset(
        cfg=Config.fromfile(dataset_cfgs[d_name]),
        device=device,
        split="val",
    )
    for d_name in dataset_cfgs.keys()
}

In [ ]:
import os


# figure directory for saving
fig_dir = "figures/examples/"
os.makedirs(fig_dir, exist_ok=True)

In [ ]:
import cv2
from avapi.visualize.snapshot import show_lidar_bev_with_boxes, show_image_with_boxes


idx_frame_eval = {"CARLA": 18, "nuScenes": 5, "UGV": 50}
camera_names = {"CARLA": "camera-0", "nuScenes": "CAM_FRONT", "UGV": "camera"}
max_ranges = {"CARLA": 70, "nuScenes": 70, "UGV": 18}

ex_v = 50
ex_h = 30
extents = {
    "CARLA": [(-ex_v, ex_v), (-ex_h, ex_h), (-10, 10)],
    "nuScenes": [(-ex_v, ex_v), (-ex_h, ex_h), (-10, 10)],
    "UGV": [(-18, 18), (-10, 10), (-10, 10)],
}

# show some examples from each data source
for d_name in idx_frame_eval:
    cfg = Config.fromfile(dataset_cfgs[d_name])
    print(f"Running over {d_name} dataset")

    # get metadata and scene dataset
    SM, seg_dataset = datasets[d_name]
    frame = idx_frame_eval[d_name]
    metadata = seg_dataset.get_metadata(frame)
    SD = SM.get_scene_dataset_by_name(metadata["scene"])
    gt_poly_avstack = seg_dataset.get_polygon_avstack(frame, SM)

    # get image and point cloud
    img = SD.get_image(
        frame=metadata["frame"],
        sensor=camera_names[d_name],
        agent=metadata["agent"],
    )
    pc = SD.get_lidar(
        frame=metadata["frame"],
        sensor=metadata["sensor"],
        agent=metadata["agent"],
        max_range=max_ranges[d_name],
    )
    objs = SD.get_objects(
        frame=metadata["frame"],
        sensor=metadata["sensor"],
        agent=metadata["agent"],
    )

    # show image and lidar bev
    cam_img_out = show_image_with_boxes(
        img=img,
        boxes=objs,
        inline=True,
        show=True,
        return_image=True
    )
    cv2.imwrite(
        os.path.join(fig_dir, f"{d_name}_frame_{frame}_image.png"),
        cv2.cvtColor(cam_img_out, cv2.COLOR_BGR2RGB),
    )

    # show lidar pc and save result
    pc_img_out = show_lidar_bev_with_boxes(
        pc=pc,
        boxes=objs,
        background_color="black",
        fov=gt_poly_avstack,
        fov_filled=True,
        fov_filled_alpha=0.5,
        return_image=True,
        show=True,
        extent=extents[d_name],
        rescale=False,
        scale_return_image=False,
    )
    cv2.imwrite(
        os.path.join(fig_dir, f"{d_name}_frame_{frame}_pc_with_fov.png"),
        cv2.cvtColor(pc_img_out, cv2.COLOR_BGR2RGB),
    )

In [ ]:
SD.get_camera_frame(0).shape